### Import Libraries and datasets

In [5]:
import timeit
start_time = timeit.default_timer()  #timestamp to calculate total runtime

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import scipy.stats as ss

import folium
from folium import plugins

%matplotlib inline
pd.set_option("display.max_columns", None)  # prevent output trancation
sns.set() 

from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
crashes = pd.read_csv(
    'crashes.sample2020.csv',
    parse_dates=["CRASH_DATE", "CRASH_DATE_EST_I", "DATE_POLICE_NOTIFIED"],
    low_memory=False,)
vehicles = pd.read_csv("vehicles.sample.csv",
                       parse_dates=["CRASH_DATE"],
                       low_memory=False)
people = pd.read_csv("people.sample.csv",
                     parse_dates=["CRASH_DATE"],
                     low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: 'crashes.sample2020.csv'

In [8]:
# split the data into train and test by 80/20
# train_size = int(len(data) * .8)
# print ("Train size %d" % train_size)
# print ("Test size %d" % (len(data) - train_size))

In [9]:
# Create bag of words matrices 
# from preprocess import Textpreprocessor
# train_qs = data['text'].value[:train_size]
# test_qs = data['text'].value[train_size:]

# VOCAB_SIZE = 400
# preprocessor = TextPreprocessor(VOCAB_SIZE)
# preprocessor.create_tokenizer(train_qs)

# body_train = preprocessor.transform_text(train_qs)
# body_test = processor.transform_text(test.qs)

In [ ]:
# Preview training data
# print(len(body_train[0]))
# print(body_train[0])